In [42]:
import tensorflow as tf
from onnx_tf.frontend import tensorflow_graph_to_onnx_model

In [53]:
import os, argparse

import tensorflow as tf

# The original freeze_graph function
# from tensorflow.python.tools.freeze_graph import freeze_graph 

#dir = os.path.dirname(os.path.realpath(__file__))

def freeze_graph(model_dir, output_node_names):
    """Extract the sub graph defined by the output nodes and convert 
    all its variables into constant 
    Args:
        model_dir: the root folder containing the checkpoint state file
        output_node_names: a string, containing all the output node's names, 
                            comma separated
    """
    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exist. Please specify an export "
            "directory: %s" % model_dir)

    if not output_node_names:
        print("You need to supply the name of a node to --output_node_names.")
        return -1
    
    # Prints all nodes in graph for debugging
    #[print(n.name) for n in tf.get_default_graph().as_graph_def().node]

    # We retrieve our checkpoint fullpath
    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path
    
    # We precise the file fullname of our freezed graph
    absolute_model_dir = "/".join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + "/frozen_model.pb"

    # We clear devices to allow TensorFlow to control on which device it will load operations
    clear_devices = True

    # We start a session using a temporary fresh Graph
    with tf.Session(graph=tf.Graph()) as sess:
        # We import the meta graph in the current default Graph
        saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)

        # We restore the weights
        saver.restore(sess, input_checkpoint)

        # We use a built-in TF helper to export variables to constants
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess, # The session is used to retrieve the weights
            tf.get_default_graph().as_graph_def(), # The graph_def is used to retrieve the nodes 
            output_node_names.split(",") # The output node names are used to select the useful nodes
        ) 

        # Finally we serialize and dump the output graph to the filesystem
        with tf.gfile.GFile('models/iris/' + output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph." % len(output_graph_def.node))

    return output_graph_def

#parser = argparse.ArgumentParser()
#parser.add_argument("--model_dir", type=str, default="", help="Model folder to export")
#parser.add_argument("--output_node_names", type=str, default="", help="The name of the output nodes, comma separated.")
#args = parser.parse_args()

#freeze_graph(args.model_dir, args.output_node_names)

freeze_graph("models\iris", "dnn/head/predictions/class_ids,dnn/head/predictions/probabilities") # Runs freezer function

#if __name__ == '__main__':
#    parser = argparse.ArgumentParser()
#    parser.add_argument("--model_dir", type=str, default="", help="Model folder to export")
#    parser.add_argument("--output_node_names", type=str, default="", help="The name of the output nodes, comma separated.")
#    args = parser.parse_args()

#    freeze_graph(args.model_dir, args.output_node_names)

/frozen_model.pb
INFO:tensorflow:Restoring parameters from models\iris\model.ckpt-4000
INFO:tensorflow:Froze 6 variables.
INFO:tensorflow:Converted 6 variables to const ops.
78 ops in the final graph.


node {
  name: "enqueue_input/random_shuffle_queue"
  op: "RandomShuffleQueueV2"
  attr {
    key: "capacity"
    value {
      i: 1000
    }
  }
  attr {
    key: "component_types"
    value {
      list {
        type: DT_INT64
        type: DT_DOUBLE
        type: DT_DOUBLE
        type: DT_DOUBLE
        type: DT_DOUBLE
        type: DT_INT32
      }
    }
  }
  attr {
    key: "container"
    value {
      s: ""
    }
  }
  attr {
    key: "min_after_dequeue"
    value {
      i: 250
    }
  }
  attr {
    key: "seed"
    value {
      i: 0
    }
  }
  attr {
    key: "seed2"
    value {
      i: 0
    }
  }
  attr {
    key: "shapes"
    value {
      list {
        shape {
        }
        shape {
        }
        shape {
        }
        shape {
        }
        shape {
        }
        shape {
        }
      }
    }
  }
  attr {
    key: "shared_name"
    value {
      s: ""
    }
  }
}
node {
  name: "random_shuffle_queue_DequeueMany/n"
  op: "Const"
  attr {
    key: "

In [ ]:
# Proto and graph freezing defined from
# https://github.com/onnx/tutorials/blob/master/tutorials/OnnxTensorflowExport.ipynb

# Bazel command below should find output node:
# bazel build C:\Users\camp-pry\AppData\Local\Continuum\anaconda3\envs\tensorflow_iris\Lib\site-packages\tensorflow\tools\graph_transforms:summarize_graph
#    bazel-bin/tensorflow/tools/graph_transforms/summarize_graph 
#    --in_graph=C:\Users\camp-pry\IdeaProjects\TensorflowTest\src\main\java\com\TensorflowTest\TensorflowTest\Training\stored_model\1533207083\saved_model.pb

# Bazel command below should generate new frozen model:
#bazel build tensorflow/python/tools:freeze_graph
#bazel-bin/tensorflow/python/tools/freeze_graph \
#    --input_graph=/home/mnist-tf/graph.proto \
#    --input_checkpoint=/home/mnist-tf/ckpt/model.ckpt \
#    --output_graph=/tmp/frozen_graph.pb \
#    --output_node_names=fc2/add \
#    --input_binary=True

In [15]:
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Exposes the Python wrapper for graph transforms."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# pylint: disable=unused-import,wildcard-import, line-too-long
from tensorflow.core.framework import graph_pb2
from tensorflow.python.framework import errors
from tensorflow.python.pywrap_tensorflow import TransformGraphWithStringInputs
from tensorflow.python.util import compat


def TransformGraph(input_graph_def, inputs, outputs, transforms):
  """Python wrapper for the Graph Transform Tool.

  Gives access to all graph transforms available through the command line tool.
  See documentation at https://github.com/tensorflow/tensorflow/blob/master/tensorflow/tools/graph_transforms/README.md
  for full details of the options available.

  Args:
    input_graph_def: GraphDef object containing a model to be transformed.
    inputs: List of node names for the model inputs.
    outputs: List of node names for the model outputs.
    transforms: List of strings containing transform names and parameters.

  Returns:
    New GraphDef with transforms applied.
  """

  input_graph_def_string = input_graph_def.SerializeToString()
  inputs_string = compat.as_bytes(",".join(inputs))
  outputs_string = compat.as_bytes(",".join(outputs))
  transforms_string = compat.as_bytes(" ".join(transforms))
  with errors.raise_exception_on_not_ok_status() as status:
    output_graph_def_string = TransformGraphWithStringInputs(
        input_graph_def_string, inputs_string, outputs_string,
        transforms_string, status)
  output_graph_def = graph_pb2.GraphDef()
  output_graph_def.ParseFromString(output_graph_def_string)
  return output_graph_def


In [54]:
# Converts frozen model to ONNX format
with tf.gfile.GFile("models/iris/frozen_model.pb", "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    onnx_model = tensorflow_graph_to_onnx_model(graph_def,
                                     "dnn/head/predictions/class_ids", #This is the output node
                                     #opset=6,
                                     graph_name="graph",
                                     ignore_unimplemented=True)

    file = open("models/iris/graph.onnx", "wb")
    file.write(onnx_model.SerializeToString())
    file.close()

c:\users\camp-pry\appdata\local\continuum\anaconda3\envs\tensorflow_iris\lib\site-packages\onnx_tf\frontend.py:533: UserWarning: RandomShuffleQueueV2 op is not implemented.
  exception(message)


ValueError: You passed in an iterable attribute but I cannot figure out its applicable type.